<a href="https://colab.research.google.com/github/YONGGIJUNG/actual-deeplearning/blob/master/CNN_model_building_caltech101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Data Preprocessing

In [ ]:
from keras_preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(rescale=1. / 255, validation_split=0.2)
#each pixcel size will be rescaled by 255, which also means that 'rgb' will be mearsured from 0~1

In [ ]:
batch_size = 16
input_image_size = (200,200)
#Since each data size in caltech101 data set all differs

train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/caltech101.zip (Unzipped Files)',
    target_size = input_image_size,
    batch_size = batch_size,
    class_mode = "categorical", #Since we are working on classficiation in 101 types of imange, it  is not binary.
    subset = "training"
)

validation_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/caltech101.zip (Unzipped Files)',
    target_size = input_image_size,
    batch_size = batch_size,
    class_mode = "categorical", #Since we are working on classficiation in 101 types of imange, it  is not binary.
    subset = "validation"
)

Found 6982 images belonging to 101 classes.
Found 1695 images belonging to 101 classes.


##Convolutional Neural Network modeling: layer - Conv2D class


In [ ]:
#Convolutional Neural Network modeling: layer - Conv2D class
from keras.layers import Dense, Input, Activation, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.models import Model, Sequential

In [ ]:
#adding Conv2D layer on sequential
model = Sequential()
model.add(Conv2D(32,(3,3), activation= "relu",input_shape=(200,200,3)))
# 32 filters, 3x3 kernal, activaiton function: relu, and 200, 200 image size and rgb(3colors, not black&white)
model.add(MaxPooling2D(pool_size=(2,2)))
#adding pulling layer to get max value, so pulling filter wil lbe(2,2)



In [ ]:
#adding more Convolution & pulling layers continuously : 2 times
model.add(Conv2D(32,(3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

#FYI : relu 가 sigmoid 함수 보다 일반적으로 성능이 좋다, 하지만 마지막 레이어서는 sigmoid 함수로 설정하면 좋은 성능이 나올수도??..

In [ ]:
#setting up FC(Fully connected layer) : steps from 2 dimentional => 1 dimentional, by using Flatten layer
model.add(Flatten())
model.add(Dense(1024)) # from Dense => make FC 
model.add(Activation('sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(101)) # Since we have 101 classfication, we set it up like this again
model.add(Activation('softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 198, 198, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 99, 99, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 97, 97, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 48, 48, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 46, 46, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 23, 23, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 33856)             0

##Train data set

In [ ]:
#Use compile => to optimize loss funtion by using adam among the list of Adam, SGD, RMSprop.

from keras import optimizers

model.compile(
    optimizer = optimizers.Adam(),
    loss = "categorical_crossentropy",
    metrics = ['acc'])

In [ ]:

train_generator.samples

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.77 µs


6982

In [ ]:
# Early stop options
%time
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(patience=15, mode = 'auto',
                               monitor='val_acc')
hist = model.fit_generator(train_generator,
                           steps_per_epoch = train_generator.samples // batch_size,
                           validation_data = validation_generator,
                           epochs = 100,
                           callbacks=[early_stopping])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
 15/436 [>.............................] - ETA: 1:11:03 - loss: 5.0020 - acc: 0.0469

KeyboardInterrupt: ignored

##Visualization

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

fig, loss_ax = plt.subplots()
acc_ax = loss_ax.twinx()

loss_ax.plot(hist.history['loss'], 'y', label = 'train_loss')
loss_ax.plot(hist.history['val_loss'], 'g', label = 'validation_loss')

acc_ax.plot(hist.history['acc'], 'b', label ='train_acc')
acc_ax.plot(hist.history['val_acc'], 'r', label ='validation_acc')

plt.legend(['train acc', 'validation acc'])
plt.show()
